In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torch
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

/home/uj-user/Yo/hybrid-ltm/ltm-venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = "/home/uj-user/Yo/hybrid-ltm/data_eval/context-fms/MS_DG/test.jsonl"
data_path = "/home/uj-user/Yo/hybrid-ltm/data_eval/context-fms/MS_DG/test_last_session.jsonl"
# model_name = "facebook/blenderbot-3B"
# model_name = "gpt2"
model_name = "jihyoung/rebot-generation"
MODEL_MAX_LEN=1024
TARGET_LEN = 128
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [3]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

In [4]:
summarizer = pipeline("summarization",
                                   model=AutoModelForSeq2SeqLM.from_pretrained("jihyoung/rebot-summarization").to(device),
                                   tokenizer=AutoTokenizer.from_pretrained("jihyoung/rebot-summarization"), device=0)

In [5]:
dataset = pd.read_json(data_path, lines=True)
dataset

,input
0,```\nDialogue Session #3:\nDoctor:Hello Patien...
1,```\nDialogue Session #3:\nNeighbors A:Hey the...
2,"```\nDialogue Session #4:\nCo-workers A:Hey, B..."
3,"```\nDialogue Session #2:\nMentee:Mentor, I ha..."
4,```\nDialogue Session #5:\nClassmates A:Sorry ...
...,...
75,```\nDialogue Session #4:\nHusband:You know wh...
76,"```\nDialogue Session #4:\nClassmates B:Hey, C..."
77,```\nDialogue Session #3:\nClassmates A:I've b...
78,```\nDialogue Session #2:\nHusband:I found a s...


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, truncation_side='left', use_fast=False)
# tokenizer.add_special_tokens({'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'})
tokenizer

BartTokenizer(name_or_path='jihyoung/rebot-generation', vocab_size=50265, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='left', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [7]:
import re
dataset['prep_input'] = dataset['input'].map(lambda x: '```'+ '```'.join(x.split('```')[1:-1]) +'```')
text_list = dataset['prep_input'][0].split('Dialogue Session #')[1:]
result_list = [re.sub(r'^\d+:\s*', '', line) for line in text_list] # #1 숫자 제거
result_list = [re.sub(r'(\b[^\s:]+):', r'<\1>', line) for line in result_list] # : -> <> 형식 변경
result_list = [line.replace('\n', ' ').replace('```', '').strip() for line in result_list] # 불필요 개행문자, 공백제거
result_list

["<Doctor>Hello Patient, how can I assist you today? <Patient>Hi Doctor, I've been thinking of traveling to a new destination. <Doctor>That sounds like a great idea. Have you decided on where you want to go? <Patient>I haven't decided yet, but I'm looking for recommendations. <Doctor>Well, I recently went to Japan and it was a wonderful experience. The culture, food, and people were amazing. <Patient>That sounds interesting. How was the travel experience for you? <Doctor>It was a long flight, but I made sure to stay hydrated and take breaks to stretch. Have you traveled internationally before? <Patient>Yes, a few years ago I went to Europe. <Doctor>That's great. Did you have any issues with your health during that trip? <Patient>No, everything was fine. Although, I did experience some jet lag when I arrived back home. <Doctor> ###"]

In [8]:
ans_jsons = []
single_session = []

for i, dialogue_lines in enumerate(dataset['prep_input']):
    # relationship
    relationship = "Speaker A and Speaker B"
    role = "Speaker A"
    if ' and ' in relationship:
        rel = relationship.strip().split(' and ')
        rel.remove(role)
        speaker = [role, rel[0]]
    else:
        speaker = [f'{relationship} A', f'{relationship} B']
    sequence = "<relationship> {}".format(relationship)

    # previous dialouge extraction
    text_list = dialogue_lines.split('Dialogue Session #')[1:]
    session_list = [re.sub(r'^\d+:\s*', '', line) for line in text_list] # #1 숫자 제거
    session_list = [re.sub(r'(\b[^\s:]+):', r'<\1>', line) for line in session_list] # : -> <> 형식 변경
    session_list = [line.replace('\n', ' ').replace('```', '').strip() for line in session_list] # 불필요 개행문자, 공백제거
    
    # previous dialogue summary
    for single_session in session_list[:-1]:
        session_event = summarizer(single_session)[0]['summary_text']
        single_session = []
        time_interval = "A few hours after"
        sequence = "{} <{}> {}".format(sequence, time_interval, session_event)
        
    # now session generate
    single_session = session_list[-1][:-4] # last session
    input_text = "{} [now] {}".format(sequence, single_session)
    print('input_text:', input_text)
    inputs = tokenizer([input_text], max_length=MODEL_MAX_LEN, truncation=True, return_tensors='pt').to(device)#.input_ids
    
    output_ids = model.generate(inputs["input_ids"], max_new_tokens=TARGET_LEN, temperature=1.0, top_p=.95,do_sample=True)
    outputs = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False).strip()
                
    # print('input:\n', tokenizer.decode(inputs.input_ids[0]))
    # print('output:\n', outputs)
    # if i%3==1:
    #     break
    
    ans_jsons.append(
        {
            "question_id": i,
            "text": outputs,
            "model_id": model_name,
            "metadata": {},
        }
    )

input_text: <relationship> Speaker A and Speaker B [now] <Doctor>Hello Patient, how can I assist you today? <Patient>Hi Doctor, I've been thinking of traveling to a new destination. <Doctor>That sounds like a great idea. Have you decided on where you want to go? <Patient>I haven't decided yet, but I'm looking for recommendations. <Doctor>Well, I recently went to Japan and it was a wonderful experience. The culture, food, and people were amazing. <Patient>That sounds interesting. How was the travel experience for you? <Doctor>It was a long flight, but I made sure to stay hydrated and take breaks to stretch. Have you traveled internationally before? <Patient>Yes, a few years ago I went to Europe. <Doctor>That's great. Did you have any issues with your health during that trip? <Patient>No, everything was fine. Although, I did experience some jet lag when I arrived back home. <Doctor>
input_text: <relationship> Speaker A and Speaker B [now] Neighbors <A>Hey there, have you ever considered 

In [9]:
filename = f"predictions/{model_name}_{data_path.split('/')[-2]}.json"
save_path = os.path.dirname(filename)
os.makedirs(save_path, exist_ok=True)
df_ans_json = pd.DataFrame(ans_jsons)
df_ans_json.to_json(filename, orient='records', lines=True)